# Introduction to LLM Agents with LangChain
---
## 🛠️ Part 1 Solutions: Tools

**Content:**
1. [Exercise 1.1 (a): Explore tool parameters](#1a)
2. [Exercise 1.1 (b): Run tool and explore output](#1b)
3. [Exercise 1.2 (a): Build your own Weather tool](#2a)
4. [Exercise 1.2 (b): Build your own Image tool](#2b)

In [1]:
import sys
import os
current_dir = os.path.dirname(os.path.abspath('.'))
folder_b_path = os.path.join(current_dir, 'helper_functions')
sys.path.append(current_dir)

import requests
from helper_functions.keys import WEATHER_KEY, HUGGING_FACE_KEY
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import StructuredTool
from PIL import Image
import io

---

#### Exercise 1.1 (a): Explore tool parameters <a id='1a'></a>

**Task:**  
Use the methods ``name``, ``description``, ``args``, ``return_direct``, ``metadata`` to familiarize yourself with the parameters of the tool. What is the meaning of the different parameters?

**Background:** 
Each tool is a ``BaseTool`` class object, you can find its definition [here](https://api.python.langchain.com/en/latest/tools/langchain_core.tools.BaseTool.html#langchain_core.tools.BaseTool).

In [2]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

**SOLUTION:**

In [3]:
print("Name: ", wiki_tool.name)
print("Description: ", wiki_tool.description)
print("Input argument schema: ", wiki_tool.args)
print("Return output to user? ", wiki_tool.return_direct)
print("Metadata: ", wiki_tool.metadata)

Name:  wikipedia
Description:  A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.
Input argument schema:  {'query': {'title': 'Query', 'description': 'query to look up on wikipedia', 'type': 'string'}}
Return output to user?  False
Metadata:  None


---

#### Exercise 1.1 (b): Run tool and explore output <a id='1b'></a>

**TASK:**
* Use the ``.run(tool_input)`` method to execute the tool. The ``tool_input`` is the search term that you'd like to query wikipedia with.
* [Optional] Check out the arguments of the WikipediaAPIWrapper [here](https://api.python.langchain.com/en/latest/utilities/langchain_community.utilities.wikipedia.WikipediaAPIWrapper.html) and modify its parameters above. How does the output change? 

**SOLUTION:**

In [8]:
tool_input = 'Amsterdam'

print(wiki_tool.run(tool_input))

Page: Amsterdam
Summary: Amsterdam ( AM-stər-dam, UK also  AM-stər-DAM, Dutch: [ˌɑmstərˈdɑm] ; literally, "Dam in the Amstel") is the capital and most populated city of the Netherlands. It has a population of 921,402 within the city proper, 1,457,018 in the urban area and 2,480,394 in the metropolitan area. Located in the Dutch province of North Holland, Amsterdam is colloquially referred to as the "Venice of the North", for its large number of canals, now a UNESCO World Heritage Site.
Amsterdam was founded at the mouth of the Amstel River, which was dammed to control flooding. Originally a small fishing village in the 12th century, Amsterdam became a major world port during the Dutch Golden Age of the 17th century, when the Netherlands was an economic powerhouse. Amsterdam was the leading centre for finance and trade, as well as a hub of secular art production. In the 19th and 20th centuries, the city expanded and new neighborhoods and suburbs were built. The city has a long tradition

---

#### Exercise 1.2 (a): Build your own Weather tool <a id='2a'></a>
The goal is to build a tool that extracts weather information from the weather site visualcrossing.com. You typically need an API key to extract information from a website. In this example we provide you with the API key. 

**TASK:** 
- Build the tool by defining the input parameters and the descriptions. The tool function is already provided to you. 
- Turn function, description and input parameters into a tool through ``StructuredTool.from_function()``.
- Test if the tool gives an output.

**SOLUTION:**

In [9]:
# Define the function
def extract_city_weather(city:str)->str:

    # Build the API URL
    url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{city}?key={WEATHER_KEY}&unitGroup=metric"

    response = requests.get(url)

    # extract response
    if response.status_code == 200:
        data = response.json()
        current_temp = data['days'][0]['temp']
        output = f"Current temperature in {city}: {current_temp}°C"
    else:
        output = f"Error: {response.status_code}"

    return output

# Input parameter definition
class WeatherInput(BaseModel):
    city: str = Field(description="City name")


# the tool description
description: str = (
        "Allows to extract the current temperature in a specific city"
    )

# fuse the function, input parameters and description into a tool. 
weather_tool = StructuredTool.from_function(
    func=extract_city_weather,
    name="weather",
    description=description,
    args_schema=WeatherInput,
    return_direct=False,
)

# test the output of the tool
print(weather_tool.run('Amsterdam'))

Current temperature in Amsterdam: 15.4°C


---

#### Exercise 1.2 (b): Build your own Image tool <a id='2b'></a>
The goal is to build a tool that generates an image based on a given prompt. **That means that later when you can build the Agent you can have an LLM that only outputs text, but also images!**  

To develop this, you can make use of `Stable Diffusion v2-1`, text-to-image model available on HuggingFace. We provided a HuggingFace token (that you loaded in the start). 

**TASK:** 
- Build the tool by defining the input parameters and the descriptions. The tool function is already provided to you. 
- Turn function, description and input parameters into a tool through ``StructuredTool.from_function()``.
- Test if the tool gives an output.

**SOLUTION:**

In [11]:
def text_to_image(payload:str):

    # Call the text-to-image API with the provided palaod
    API_URL = "https://api-inference.huggingface.co/models/stabilityai/stable-diffusion-2-1"
    headers = {"Authorization": f"Bearer {HUGGING_FACE_KEY}"}

    def query(payload):
        response = requests.post(API_URL, headers=headers, json=payload)
        return response.content
    
    image_bytes = query({
        "inputs": payload,
    })
    image = Image.open(io.BytesIO(image_bytes))
    
    # Resize the image
    new_size = (400, 400)  # Example new size (width, height)
    resized_image = image.resize(new_size)

    # Save the resized image to a file
    image_path = f'images/image_{payload.replace(" ", "_")}.jpg'
    resized_image.save(image_path)
    
    # Return the path to the saved image
    return f'{image_path} '
        


# Input parameter definition
class ImageInput(BaseModel):
    payload: str = Field(description="What should be converted into image")


# the tool description
images_tool_description: str = (
       "Generate an image based on the input text and return its path"
    )

# fuse the function, input parameters and description into a tool. 
image_tool = StructuredTool.from_function(
        func=text_to_image,
        name="create_image",
        description=images_tool_description,
        args_schema=ImageInput,
        return_direct=False,
)
image_tool.run('cat in a blue box')

'images/image_cat_in_a_blue_box.jpg '

---

## 🤖 Part 2 Solutions: Agents


**Content:**

1. [Exercise 2.1: Explore the agent's output](#1)
2. [Exercise 2.2 : Build your own agent](#2)
3. [Exercise 2.3: Invoke as many tools as you can](#3)
4. [Exercise 2.4 : Optimize the agent prompt](#4)

In [12]:
import sys
import os
current_dir = os.path.dirname(os.path.abspath('.'))
folder_b_path = os.path.join(current_dir, 'helper_functions')
sys.path.append(current_dir)

from helper_functions.keys import OPENAI_KEY
from helper_functions.tools import my_own_wiki_tool, weather_tool, image_tool

from langchain.agents import create_tool_calling_agent # set up the agent
from langchain.agents import AgentExecutor # execute agent
from langchain_openai import ChatOpenAI # call openAI as agent llm
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

---

#### Exercise 2.1: Explore the agent's output <a id='1'></a>

**TASK:**
In the examples below, read the output to observe how the Agent reasons and decides to use a different tool based on the context.

In [13]:
# Agent

# Load Tools
tools = [my_own_wiki_tool, weather_tool, image_tool]

# Load LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, api_key=OPENAI_KEY)


# Get the prompt to use - you can modify this! With this you let the agent know what its purpose is.
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a nice assistant"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

# Define  the agent (load the LLM and the list of tools)
agent = create_tool_calling_agent(llm = llm, tools = tools, prompt = prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

print("Your agent is ready.")


Your agent is ready.


**SOLUTION:**

In [14]:
question_1 = "Where is Amsterdam?"


print(f"Question 1: {question_1}")
agent_executor.invoke({"input": question_1})

Question 1: Where is Amsterdam?


> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Amsterdam'}`


Page: Amsterdam
Summary: Amsterdam ( AM-stər-dam, UK also  AM-stər-DAM, Dutch: [ˌɑmstərˈdɑm] ; literally, "Dam in the Amstel") is the capital and most populated city of the Netherlands. It has a population of 921,402 within the city proper, 1,457,018 in the urban area and 2,480,394 in the metropolitan area. Located in the Dutch province of North Holland, Amsterdam is colloquially referred to as the "Venice of the North", for its large number of canals, now a UNESCO World Heritage Site.
Amsterdam was founded at the mouth of the Amstel River, which was dammed to control flooding. Originally a small fishing village in the 12th century, Amsterdam became a major world port during the Dutch Golden Age of the 17th century, when the Netherlands was an economic powerhouse. Amsterdam was the leading centre for finance and trade, as well as a hub of secular art production

{'input': 'Where is Amsterdam?',
 'output': 'Amsterdam is the capital and most populated city of the Netherlands. It is located in the Dutch province of North Holland. The city is situated at the mouth of the Amstel River, which was dammed to control flooding. Amsterdam is known for its historic canals, artistic heritage, and vibrant cultural scene. It is often referred to as the "Venice of the North" due to its extensive canal system, which is a UNESCO World Heritage Site.'}

In [15]:
question_2 = "What is the current temperature in Amsterdam?"

print(f"Question 2: {question_2}")
agent_executor.invoke({"input": question_2})

Question 2: What is the current temperature in Amsterdam?


> Entering new AgentExecutor chain...

Invoking: `weather` with `{'city': 'Amsterdam'}`


Current temperature in Amsterdam: 15.4°CThe current temperature in Amsterdam is 15.4°C.

> Finished chain.


{'input': 'What is the current temperature in Amsterdam?',
 'output': 'The current temperature in Amsterdam is 15.4°C.'}

In [16]:
question_3 = "What should I visit in Amsterdam? Show me an photo"

print(f"Question 1: {question_3}")
agent_executor.invoke({"input": question_3})

Question 1: What should I visit in Amsterdam? Show me an photo


> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Tourist attractions in Amsterdam'}`


Page: List of tourist attractions in Amsterdam
Summary: Amsterdam, one of Europe's capitals, has many attractions for visitors. The city's most famous sight is the 17th-century canals of Amsterdam (in Dutch: grachtengordel), located in the heart of Amsterdam, have been added to the UNESCO World Heritage List.
Invoking: `create_image` with `{'payload': 'Tourist attractions in Amsterdam'}`


images/image_Tourist_attractions_in_Amsterdam.jpg Amsterdam offers a variety of attractions for visitors. One of the most famous sights is the 17th-century canals, known as the "grachtengordel," which are located in the heart of the city and have been added to the UNESCO World Heritage List.

Here is an image of some tourist attractions in Amsterdam:

![Tourist attractions in Amsterdam](images/image_Tourist_attractions_in_

{'input': 'What should I visit in Amsterdam? Show me an photo',
 'output': 'Amsterdam offers a variety of attractions for visitors. One of the most famous sights is the 17th-century canals, known as the "grachtengordel," which are located in the heart of the city and have been added to the UNESCO World Heritage List.\n\nHere is an image of some tourist attractions in Amsterdam:\n\n![Tourist attractions in Amsterdam](images/image_Tourist_attractions_in_Amsterdam.jpg)'}

---

#### Exercise 2.2 : Build your own agent  <a id='2'></a>
The goal is to build an agent that uses the tools you previously developed. Feel free to also use the pre-made tools, available at `helper_functions/tools.py`

**TASK**: 
- A template for defining an agent and an API key are already provided to you. Build an agent by using a list of tools, and the LLM `gpt-4o-mini`. A template for this agent follows below.
- Test if the agent gives an output.
- Observe how the output changes if you provide less tools in your list of tools.

**SOLUTION:**

In [18]:
# Component 1 (Tools): Load Tools
tools = [
    my_own_wiki_tool, weather_tool, image_tool
]

# Component 2 (LLM): Load LLM
llm = ChatOpenAI(model="gpt-4o-mini", 
                 temperature=0, 
                 api_key=OPENAI_KEY)

# Component 3 (Prompt): Let the agent know what its purpose is. For now, let's keep it as is.
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a nice assistant"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

# Define the agent and agent executor (load the LLM, the list of tools, and the prompt (descripiton))

agent = create_tool_calling_agent(
    llm = llm, tools = tools, prompt = prompt
    )
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True
)

print("Your agent is ready.")


Your agent is ready.


---

#### Exercise 2.3: Invoke as many tools as you can  <a id='3'></a>

**TASK:**
Try various questions to call the agent and follow the generated reasoning process in the response. The goal is to call the agent in a way thay it will use as many tools as possible. **Let's see who can reach the highest number of tools used with a single prompt!**

**SOLUTION:** (invoking 2 tools: Wiki and Weather. Can you reach more?)

In [19]:
question = """
Where is Amsterdam and what is the weather like?
"""

print(f"Question: {question}")
agent_executor.invoke({"input": question})

Question: 
Where is Amsterdam and what is the weather like?



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Amsterdam'}`


Page: Amsterdam
Summary: Amsterdam ( AM-stər-dam, UK also  AM-stər-DAM, Dutch: [ˌɑmstərˈdɑm] ; literally, "Dam in the Amstel") is the capital and most populated city of the Netherlands. It has a population of 921,402 within the city proper, 1,457,018 in the urban area and 2,480,394 in the metropolitan area. Located in the Dutch province of North Holland, Amsterdam is colloquially referred to as the "Venice of the North", for its large number of canals, now a UNESCO World Heritage Site.
Amsterdam was founded at the mouth of the Amstel River, which was dammed to control flooding. Originally a small fishing village in the 12th century, Amsterdam became a major world port during the Dutch Golden Age of the 17th century, when the Netherlands was an economic powerhouse. Amsterdam was the leading centre for finance and trade, as well as a 

{'input': '\nWhere is Amsterdam and what is the weather like?\n',
 'output': '### Location of Amsterdam:\nAmsterdam is the capital and most populated city of the Netherlands. It is located in the Dutch province of North Holland. The city is known for its extensive canal system, which has earned it the nickname "Venice of the North." Amsterdam was founded at the mouth of the Amstel River and has a rich history dating back to the 12th century. It became a major world port during the Dutch Golden Age in the 17th century and is now a cultural and financial hub.\n\n### Current Weather in Amsterdam:\nThe current temperature in Amsterdam is 15.4°C.'}

---

#### Exercise 2.4 : Optimize the agent prompt  <a id='4'></a>
So far we played around with the provided LLM and list of tools. Now let's look into the 3rd component: the Agent prompt. 

**TASK:**
- Modify the agent prompt and observe the difference in the output. 

**SOLUTION:**

In this solution we assume you are building an agent for a travel agency, and you want the agent to help you motivate people to visit a city.

In [20]:
# Component 1 (Tools): Load Tools from Exercise 1
tools = tools 

# Component 2 (LLM): Load LLM form Exercise 1
llm = llm

your_prompt = """"
    You are a helpful assisstant, trying to motivate people to go on holiday.
"""

# Component 3 (Prompt): Create your own prompt to instruct the Agent about its purpose.
prompt = ChatPromptTemplate.from_messages([
    ("system", your_prompt),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])
prompt.messages


# Define the agent and agent executor (load the LLM, the list of tools, and the prompt (descripiton))
# This is same as in Exercise 1
agent = create_tool_calling_agent(
    llm = llm, tools = tools, prompt = prompt
    )
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True
)

print("Your agent is ready.")


Your agent is ready.


In [21]:
# Observe how the same question from before is answered differently with the different prompt.
print(f"Question: {question}")
agent_executor.invoke({"input": question})

Question: 
Where is Amsterdam and what is the weather like?



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Amsterdam'}`


Page: Amsterdam
Summary: Amsterdam ( AM-stər-dam, UK also  AM-stər-DAM, Dutch: [ˌɑmstərˈdɑm] ; literally, "Dam in the Amstel") is the capital and most populated city of the Netherlands. It has a population of 921,402 within the city proper, 1,457,018 in the urban area and 2,480,394 in the metropolitan area. Located in the Dutch province of North Holland, Amsterdam is colloquially referred to as the "Venice of the North", for its large number of canals, now a UNESCO World Heritage Site.
Amsterdam was founded at the mouth of the Amstel River, which was dammed to control flooding. Originally a small fishing village in the 12th century, Amsterdam became a major world port during the Dutch Golden Age of the 17th century, when the Netherlands was an economic powerhouse. Amsterdam was the leading centre for finance and trade, as well as a 

---